### Install dependencies

In [1]:
%env HDF5_USE_FILE_LOCKING=FALSE
# %env WANDB_MODE=dryrun
%env WANDB_NOTEBOOK_NAME=deepmr

env: HDF5_USE_FILE_LOCKING=FALSE
env: WANDB_NOTEBOOK_NAME=deepmr


In [2]:
import bpnet
from bpnet.cli.contrib import ContribFile
from bpnet.plot.tracks import plot_tracks, to_neg

import os
import uuid
from datetime import datetime
import seaborn as sns
import matplotlib.pyplot as plt
from IPython.display import clear_output, HTML
from pathlib import Path
import pandas as pd
import numpy as np
clear_output()

#### Optional: Setup wandb

In [3]:
import wandb

wandb.init(project="deepmr", entity="an1lam")

wandb: WARNING WANDB_NOTEBOOK_NAME should be a path to a notebook file, couldn't find deepmr
wandb: Currently logged in as: an1lam (use `wandb login --relogin` to force relogin)
wandb: wandb version 0.12.18 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


In [4]:
# config variables
n_reps = 5

# file paths
config_dir = Path('./bpnet/') 

model_config_fname = 'ChIP-nexus-default.gin'
data_config_fname = 'ChIP-nexus.dataspec.yml'

timestamp = datetime.now().strftime('%Y-%m-%d-%H-%M-%S')
project_dir = '/home/ubuntu/dev/an1lam/deepmr/'
data_dir = os.path.join(project_dir, 'dat')
output_dir = os.path.join(data_dir, f'res-bpnet-training-{timestamp}')
output_dir

'/home/ubuntu/dev/an1lam/deepmr/dat/res-bpnet-training-2022-06-18-14-11-05'

In [5]:
os.makedirs(output_dir, exist_ok=True)
os.makedirs(os.path.join(output_dir, 'output_ensemble'), exist_ok=True)

In [6]:
!cat {config_dir}/{data_config_fname}

task_specs:
  Oct4:
    tracks:
    - /home/ubuntu/dev/an1lam/deepmr/dat/bpnet-manuscript-data/data/chip-nexus/Oct4/counts.pos.bw
    - /home/ubuntu/dev/an1lam/deepmr/dat/bpnet-manuscript-data/data/chip-nexus/Oct4/counts.neg.bw
    peaks: /home/ubuntu/dev/an1lam/deepmr/dat/bpnet-manuscript-data/data/chip-nexus/Oct4/idr-optimal-set.summit.bed.gz
  Sox2:
    tracks:
    - /home/ubuntu/dev/an1lam/deepmr/dat/bpnet-manuscript-data/data/chip-nexus/Sox2/counts.pos.bw
    - /home/ubuntu/dev/an1lam/deepmr/dat/bpnet-manuscript-data/data/chip-nexus/Sox2/counts.neg.bw
    peaks: /home/ubuntu/dev/an1lam/deepmr/dat/bpnet-manuscript-data/data/chip-nexus/Sox2/idr-optimal-set.summit.bed.gz
  Nanog:
    tracks:
    - /home/ubuntu/dev/an1lam/deepmr/dat/bpnet-manuscript-data/data/chip-nexus/Nanog/counts.pos.bw
    - /home/ubuntu/dev/an1lam/deepmr/dat/bpnet-manuscript-data/data/chip-nexus/Nanog/counts.neg.bw
    peaks: /home/ubuntu/dev/an1lam/deepmr/dat/bpnet-manuscript-data/data/chip-nexus/Nanog/idr-optim

### Data stats

In [7]:
# chromsomome names of differnet peaks
!zcat {project_dir}dat/bpnet-manuscript-data/data/chip-nexus/Sox2/idr-optimal-set.summit.bed.gz \
    | cut -f 1 | sort -u

chr1
chr10
chr11
chr12
chr13
chr14
chr15
chr16
chr17
chr18
chr19
chr2
chr3
chr4
chr5
chr6
chr7
chr8
chr9
chrX
chrY


Each task (or TF) can specify a set of peaks associated with it. Here are the number of peaks per TF we will use in this tutorial:

In [8]:
tasks = ['Oct4', 'Sox2', 'Nanog', 'Klf4']

# number of peaks per task
for task in tasks:
    print(task)
    !zcat {data_dir}/bpnet-manuscript-data/data/chip-nexus/{task}/idr-optimal-set.summit.bed.gz | wc -l

Oct4
25849
Sox2
10999
Nanog
56459
Klf4
57601


## 2. Train the model

Having specified `dataspec.yml`, we are now ready to train the model with 

```
bpnet train <dataspec.yml> <output dir> [optional flags]`
```


We will use a pre-made model [bpnet9](../bpnet/premade/bpnet9.gin) as a starting point and modify a few parameters specified in the config.gin file. Specifically, we will 
- train the model only on chromosomes 16-19
- evaluate the model on chromosome 2
- use only 3 layers of dilated convolutions 
- use an input sequence length of 200 bp and accordingly lower the augmentation shift to 100 bp

In [9]:
!cat {config_dir}/{model_config_fname} 
# NOTE: test_chr will be also excluded similar to 'exclude_chr'

b_loss_weight = 0
c_loss_weight = 10
p_loss_weight = 1
filters = 64
tconv_kernel_size = 25
lr = 0.004
n_dil_layers = 9
train.batch_size = 128
merge_profile_reg = False
dataspec = 'ChIP-nexus.dataspec.yml'

batchnorm = False

padding = 'same'
seq_width = 1000

tasks = ['Oct4', 'Sox2', 'Nanog', 'Klf4']


Have a look at the original gin file of bpnet9 here: https://github.com/kundajelab/bpnet/blob/master/bpnet/premade/bpnet9-ginspec.gin. For more information on using gin files see <https://github.com/google/gin-config>. 

To track model training and evaluation, we will use [wandb](http://wandb.com/) by adding `--wandb=avsec/bpnet-demo` to `bpnet train`. You can navigate to https://app.wandb.ai/avsec/bpnet-demo to see the training progress.

Let's train!

In [10]:
# setup all the file paths
example_model_dir = os.path.join(output_dir, 'output_ensemble', '0')

In [ ]:
# Train for at most 10 epochs
for i in range(n_reps):
    # setup a new run_id (could be done automatically, but then the output directory would change)
    run_id = datetime.now().strftime("%Y-%m-%d_%H-%M-%S") + "_" + str(uuid.uuid4())
    !cd {config_dir} && bpnet train {data_config_fname} --premade=bpnet9 \
        --config={model_config_fname} {output_dir} \
        --run-id '{run_id}' \
        --override='train.epochs=20; train.seed={i}'
    # softlink the new output directory
    !rm -rf {output_dir}/output_ensemble/{i} && ln -srf {output_dir}/{run_id} {output_dir}/output_ensemble/{i}

Using TensorFlow backend.

2022-06-18 14:11:10,876 [WARNING] From /home/ubuntu/anaconda3/envs/deepmr-bpnet3/lib/python3.6/site-packages/keras/optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.

2022-06-18 14:11:11,679 [INFO] NumExpr defaulting to 4 threads.
/home/ubuntu/src/bpnet/bpnet/plot/heatmaps.py:6: MatplotlibDeprecationWarning: 
The mpl_toolkits.axes_grid1.colorbar module was deprecated in Matplotlib 3.2 and will be removed two minor releases later. Use matplotlib.colorbar instead.
  from mpl_toolkits.axes_grid1.colorbar import colorbar
INFO [06-18 14:11:13] Using gpu: 0, memory fraction: 0.45
2022-06-18 14:11:13.792797: I tensorflow/core/platform/cpu_feature_guard.cc:142] Your CPU supports instructions that this TensorFlow binary was not compiled to use: AVX2 FMA
2022-06-18 14:11:13.801700: I tensorflow/core/platform/profile_utils/cpu_utils.cc:94] CPU Frequency: 2300020000 Hz
2022-06-18 14:11:13.802015: I tensorflow/co

Epoch 1/20
2022-06-18 14:11:33.118727: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcublas.so.10.0
2022-06-18 14:11:33.301675: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcudnn.so.7
710/710 [==============================] - 287s 404ms/step - loss: 3468.4141 - Oct4/profile_loss: 847.9951 - Oct4/counts_loss: 0.4355 - Sox2/profile_loss: 472.1162 - Sox2/counts_loss: 0.4387 - Nanog/profile_loss: 1160.6593 - Nanog/counts_loss: 1.0452 - Klf4/profile_loss: 948.8722 - Klf4/counts_loss: 1.9578 - val_loss: 3355.0953 - val_Oct4/profile_loss: 832.7220 - val_Oct4/counts_loss: 0.4596 - val_Sox2/profile_loss: 465.6302 - val_Sox2/counts_loss: 0.3400 - val_Nanog/profile_loss: 1109.7782 - val_Nanog/counts_loss: 0.8928 - val_Klf4/profile_loss: 919.4166 - val_Klf4/counts_loss: 1.0624
Epoch 2/20
710/710 [==============================] - 281s 395ms/step - loss: 3285.5634 - Oct4/profile_loss:

710/710 [==============================] - 286s 403ms/step - loss: 3137.2141 - Oct4/profile_loss: 799.5456 - Oct4/counts_loss: 0.2766 - Sox2/profile_loss: 449.3933 - Sox2/counts_loss: 0.2095 - Nanog/profile_loss: 992.4969 - Nanog/counts_loss: 0.5835 - Klf4/profile_loss: 880.8759 - Klf4/counts_loss: 0.4208 - val_loss: 3181.3336 - val_Oct4/profile_loss: 806.1017 - val_Oct4/counts_loss: 0.2634 - val_Sox2/profile_loss: 456.3037 - val_Sox2/counts_loss: 0.2031 - val_Nanog/profile_loss: 1024.7459 - val_Nanog/counts_loss: 0.5896 - val_Klf4/profile_loss: 879.6465 - val_Klf4/counts_loss: 0.3975
Epoch 14/20
710/710 [==============================] - 287s 404ms/step - loss: 3132.7581 - Oct4/profile_loss: 798.9890 - Oct4/counts_loss: 0.2744 - Sox2/profile_loss: 449.1209 - Sox2/counts_loss: 0.2077 - Nanog/profile_loss: 989.7282 - Nanog/counts_loss: 0.5778 - Klf4/profile_loss: 880.2013 - Klf4/counts_loss: 0.4120 - val_loss: 3197.9494 - val_Oct4/profile_loss: 808.6577 - val_Oct4/counts_loss: 0.3042 - 

Executing:  10%|███▏                           | 3/29 [00:05<00:46,  1.80s/cell]2022-06-18 15:49:49.304394: I tensorflow/core/platform/cpu_feature_guard.cc:142] Your CPU supports instructions that this TensorFlow binary was not compiled to use: AVX2 FMA
2022-06-18 15:49:49.318914: I tensorflow/core/platform/profile_utils/cpu_utils.cc:94] CPU Frequency: 2300020000 Hz
2022-06-18 15:49:49.319320: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x55d5e20f6090 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
2022-06-18 15:49:49.319373: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): Host, Default Version
2022-06-18 15:49:49.322535: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcuda.so.1
2022-06-18 15:49:49.413484: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:983] successful NUMA node read from SysFS had negative value (-1), but there must be 

Using TensorFlow backend.

2022-06-18 15:52:09,920 [WARNING] From /home/ubuntu/anaconda3/envs/deepmr-bpnet3/lib/python3.6/site-packages/keras/optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.

2022-06-18 15:52:10,748 [INFO] NumExpr defaulting to 4 threads.
/home/ubuntu/src/bpnet/bpnet/plot/heatmaps.py:6: MatplotlibDeprecationWarning: 
The mpl_toolkits.axes_grid1.colorbar module was deprecated in Matplotlib 3.2 and will be removed two minor releases later. Use matplotlib.colorbar instead.
  from mpl_toolkits.axes_grid1.colorbar import colorbar
INFO [06-18 15:52:12] Using gpu: 0, memory fraction: 0.45
2022-06-18 15:52:12.919423: I tensorflow/core/platform/cpu_feature_guard.cc:142] Your CPU supports instructions that this TensorFlow binary was not compiled to use: AVX2 FMA
2022-06-18 15:52:12.928673: I tensorflow/core/platform/profile_utils/cpu_utils.cc:94] CPU Frequency: 2300020000 Hz
2022-06-18 15:52:12.929016: I tensorflow/co

...

710/710 [==============================] - 267s 377ms/step - loss: 3130.0343 - Oct4/profile_loss: 798.1118 - Oct4/counts_loss: 0.2813 - Sox2/profile_loss: 448.8051 - Sox2/counts_loss: 0.2079 - Nanog/profile_loss: 989.0504 - Nanog/counts_loss: 0.5826 - Klf4/profile_loss: 879.3451 - Klf4/counts_loss: 0.4003 - val_loss: 3186.9644 - val_Oct4/profile_loss: 808.8915 - val_Oct4/counts_loss: 0.2653 - val_Sox2/profile_loss: 457.4978 - val_Sox2/counts_loss: 0.1977 - val_Nanog/profile_loss: 1025.4095 - val_Nanog/counts_loss: 0.6195 - val_Klf4/profile_loss: 880.5611 - val_Klf4/counts_loss: 0.3779: 448.5438 - Sox2/counts_loss: 0.2080 - Nanog/profile_loss: 987.3706 - Nanog/counts_loss: 0.5784 - Klf4/profile_loss: 878.4356 - Klf4/count - ETA: 1:18 - loss: 3127.1820 - Oct4/profile_loss: 797.8272 - Oct4/counts_loss: 0.2808 - Sox2/profile_loss: 448.5946 - So - ETA: 56s - loss: 3130.1075 - Oct4
Epoch 13/20
710/710 [==============================] - 271s 381ms/step - loss: 3129.8504 - Oct4/profile_loss: 7

Executing:  10%|███▏                           | 3/29 [00:04<00:41,  1.61s/cell]2022-06-18 19:13:49.054585: I tensorflow/core/platform/cpu_feature_guard.cc:142] Your CPU supports instructions that this TensorFlow binary was not compiled to use: AVX2 FMA
2022-06-18 19:13:49.065272: I tensorflow/core/platform/profile_utils/cpu_utils.cc:94] CPU Frequency: 2300020000 Hz
2022-06-18 19:13:49.065710: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x55f80d0198e0 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
2022-06-18 19:13:49.065737: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): Host, Default Version
2022-06-18 19:13:49.068779: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcuda.so.1
2022-06-18 19:13:49.140202: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:983] successful NUMA node read from SysFS had negative value (-1), but there must be 

Using TensorFlow backend.

2022-06-18 19:16:02,059 [WARNING] From /home/ubuntu/anaconda3/envs/deepmr-bpnet3/lib/python3.6/site-packages/keras/optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.

2022-06-18 19:16:02,798 [INFO] NumExpr defaulting to 4 threads.
/home/ubuntu/src/bpnet/bpnet/plot/heatmaps.py:6: MatplotlibDeprecationWarning: 
The mpl_toolkits.axes_grid1.colorbar module was deprecated in Matplotlib 3.2 and will be removed two minor releases later. Use matplotlib.colorbar instead.
  from mpl_toolkits.axes_grid1.colorbar import colorbar
INFO [06-18 19:16:04] Using gpu: 0, memory fraction: 0.45
2022-06-18 19:16:04.653344: I tensorflow/core/platform/cpu_feature_guard.cc:142] Your CPU supports instructions that this TensorFlow binary was not compiled to use: AVX2 FMA
2022-06-18 19:16:04.662099: I tensorflow/core/platform/profile_utils/cpu_utils.cc:94] CPU Frequency: 2300020000 Hz
2022-06-18 19:16:04.662409: I tensorflow/co

Epoch 1/20
2022-06-18 19:16:22.880637: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcublas.so.10.0
2022-06-18 19:16:23.064291: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcudnn.so.7
710/710 [==============================] - 275s 387ms/step - loss: 3596.3567 - Oct4/profile_loss: 855.2502 - Oct4/counts_loss: 1.4632 - Sox2/profile_loss: 470.5167 - Sox2/counts_loss: 0.5724 - Nanog/profile_loss: 1281.3967 - Nanog/counts_loss: 1.7824 - Klf4/profile_loss: 943.5205 - Klf4/counts_loss: 0.7492 - val_loss: 3417.3241 - val_Oct4/profile_loss: 842.6050 - val_Oct4/counts_loss: 0.8987 - val_Sox2/profile_loss: 471.3304 - val_Sox2/counts_loss: 0.3900 - val_Nanog/profile_loss: 1152.2649 - val_Nanog/counts_loss: 1.0178 - val_Klf4/profile_loss: 921.9210 - val_Klf4/counts_loss: 0.6137
Epoch 2/20
710/710 [==============================] - 270s 381ms/step - loss: 3318.7754 - Oct4/profile_loss:

710/710 [==============================] - 298s 419ms/step - loss: 3146.7372 - Oct4/profile_loss: 800.5859 - Oct4/counts_loss: 0.2908 - Sox2/profile_loss: 449.8709 - Sox2/counts_loss: 0.2142 - Nanog/profile_loss: 998.7461 - Nanog/counts_loss: 0.6039 - Klf4/profile_loss: 882.3268 - Klf4/counts_loss: 0.4118 - val_loss: 3197.5778 - val_Oct4/profile_loss: 810.9603 - val_Oct4/counts_loss: 0.2821 - val_Sox2/profile_loss: 458.0022 - val_Sox2/counts_loss: 0.2099 - val_Nanog/profile_loss: 1031.2723 - val_Nanog/counts_loss: 0.5865 - val_Klf4/profile_loss: 882.5176 - val_Klf4/counts_loss: 0.4040- Nanog/counts_loss: 0.6139 - Klf4/profile_loss: 882.5879 - Klf4/counts_loss:  - ETA: 3:17 - loss: 3140.2654 - Oct4/profile_loss: 798.6658 - Oct4/counts_loss: 0.2869 - Sox2/profile_loss: 449.1573 - Sox2/counts_loss: 0.2157 - Nanog/profile_lo - ETA: 54s - loss: 3141.5267 - Oct4/profile_loss: 799.5938 -
Epoch 13/20
710/710 [==============================] - 292s 411ms/step - loss: 3144.3699 - Oct4/profile_lo

In [ ]:
! echo {output_dir}
! echo
! ls {output_dir}/output_ensemble/0

In [ ]:
!ls -latr {example_model_dir}/